Employee which have more than 1 Leaves and more than 3 lates

In [ ]:
select EmployeeCode from MI_ONTIME_Attendance where Late>0
and EmployeeCode in (select distinct EmployeeCode from MI_ONTIME_Leave where Year(StartDate)=2020 and month(StartDate)=12)
group by EmployeeCode
having count(EmployeeCode)>3

select * from MI_ONTIME_Leave where Year(StartDate)=2020 and month(StartDate)=12 and EmployeeCode=10
select * from MI_ONTIME_Attendance where Late>0 and EmployeeCode=11

Check Attendance by number

In [ ]:
SELECT *
FROM [dbo].[mi_ontime_attendance]
WHERE employeecode in (SELECT employeecode
from MI_Employer_Employee
where number in ('000328') ) and (intime !='1901-01-01 00:00:00.000' or outtime !='1901-01-01 00:00:00.000')
and date>'2021-01-01'
order by employeecode,date

select * from MI_ONTIME_Attendance where  date > '2021-02-01' and (InTime!='1901-01-01 00:00:00.000' or OutTime!='1901-01-01 00:00:00.000')  and EmployeeCode=96
select * from MI_ONTIME_backLog where Location=''  order by CreatedDate desc
select TextCardNumber, number, EmployeeCode from MI_Employer_Employee where TextCardNumber='10029'

select getdate()

Attendance Count in a  spacifc date

In [ ]:
select count(*) from MI_ONTIME_Attendance where (InTime !='1901-01-01 00:00:00.000' or OutLate!='1901-01-01 00:00:00.000') 
and EmployeeCode in (select distinct EmployeeCode from MI_Employer_Employee where Level2Code=1 )
and date='2021-01-26'

Insert and update LeaveType

In [1]:
select top 1 * from MI_ONTIME_LeaveType
-- update MI_ONTIME_LeaveType  set SystemLeaveTypeCode='SPS' where LeaveTypeCode=7
-- insert into MI_ONTIME_LeaveType values ('Special Sick','SpS',1,0,0,0,0,0,0,0,0,1,1,getdate(),0,'1901-01-01 00:00:00.000',0,'1901-01-01 00:00:00.000')
MI_ONTIME_LeaveAllocation

(1 row affected)

Total execution time: 00:00:00.415

LeaveTypeCode,LeaveTypeName,SystemLeaveTypeCode,PaidLeaveType,PayAllowance,LeavePayment,Unlimited,PayCutEffect,AttAllowEffect,NeedCovering,NeedAttachment,Deleted,Status,CreatedBy,CreatedDate,UpdatedBy,UpdatedDate,DeletedBy,DeletedDate
1,Annual Leave,ANN,1,0,0,0,0,0,0,0,0,1,1,2020-06-10 08:45:02.923,1,2020-12-01 07:41:42.823,0,1901-01-01 00:00:00.000


Create LeaveAllocation BackUp and Insert

In [ ]:

CREATE TABLE [dbo].[MI_ONTIME_LeaveAllocation_bak](
	[LeaveAllocationCode] [int] IDENTITY(1,1) NOT NULL,
	[LeaveTypeCode] [int] NOT NULL,
	[EmployeeCode] [int] NOT NULL,
	[LeaveYear] [int] NOT NULL,
	[Allocated] [real] NOT NULL,
	[Used] [real] NOT NULL,
	[Deleted] [bit] NOT NULL,
	[Status] [bit] NOT NULL,
	[CreatedBy] [int] NULL,
	[CreatedDate] [datetime] NULL,
	[UpdatedBy] [int] NULL,
	[UpdatedDate] [datetime] NULL,
	[DeletedBy] [int] NULL,
	[DeletedDate] [datetime] NULL
 )

Go

insert into [MI_ONTIME_LeaveAllocation_bak]
SELECT [LeaveTypeCode]
      ,[EmployeeCode]
      ,[LeaveYear]
      ,[Allocated]
      ,[Used]
      ,[Deleted]
      ,[Status]
      ,[CreatedBy]
      ,[CreatedDate]
      ,[UpdatedBy]
      ,[UpdatedDate]
      ,[DeletedBy]
      ,[DeletedDate]
  FROM [dbo].[MI_ONTIME_LeaveAllocation]

update employement type by excel

In [ ]:
update MI_Employer_Employee set EmploymentTypeCode=1 where employeecode in  (select employeecode from MI_Employer_Employee where Number='' and Deleted=0) and Deleted=0

Cloud Payproll Proccess work

In [ ]:
select EmployeeCode from MI_ONTIME_Attendance where InLate>0
and EmployeeCode in 
(select distinct EmployeeCode from MI_ONTIME_Leave where Year(StartDate)=2021 and month(StartDate)=01) 
 and Year(Date)=2021 and month(Date)=01 --and EmployeeCode=16
group by EmployeeCode
having count(EmployeeCode)>3

select @SickBalance=Allocated-(@SickPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType

SELECT e.EmployeeCode
		FROM [dbo].[MI_PROLL_ProcessEmployee] AS p
			 INNER JOIN
			 [dbo].[mi_employer_employee] AS e
			 ON p.employeecode = e.employeecode
		WHERE e.Deleted = 0 AND  e.EmployeeCode=10 and 
			  e.schemecode IN( 12 ) AND 
			  invokedby = 1 AND 
		  p.Deleted = 0 

select * from MI_ONTIME_Leave where Year(StartDate)=2020 and month(StartDate)=12 and EmployeeCode=10
select * from MI_ONTIME_Attendance where Late>0 and EmployeeCode=17

select * from MI_Employer_Employee where EmployeeCode=11
select * from MI_EPRF_AutoLeaveDeduction where EmployeeCode=11
delete from MI_EPRF_AutoLeaveDeduction  where EmployeeCode=11
select * from MI_ONTIME_LeaveAllocation where leaveallocationcode=6
update MI_ONTIME_LeaveAllocation  set used=2 where leaveallocationcode=6
select * from my_log order by CreatedDate desc  

/****** Object:  StoredProcedure [dbo].[PRC_PROLL_ProcessPayroll]    Script Date: 1/29/2021 6:19:22 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
--select * from MI_PROLL_PayPeriod
--[PRC_PROLL_ProcessPayroll]  2020,3,953,12,1,1
ALTER PROCEDURE [dbo].[PRC_PROLL_ProcessPayroll] --[PRC_PROLL_ProcessPayroll] 2020,3,950,12,1,1
(
      @Year int,
	  @PayPeriodType int,
	  @PayPeriodCode int,
	  @SchemeCode int,
	  @TemplateCode int,
	  @InvokedBy int
)
AS

DECLARE @Result int;
set dateformat ymd
Declare @SDate datetime
Declare @EDate datetime
Declare @LSDate datetime
Declare @LEDate datetime
Declare @cnt int
Declare @Days Decimal(19,4)
Declare @LCode int
Declare @EmpNo varchar(50)
Declare @OThrs Decimal(19,4)
Declare @ArrearsOThrs Decimal(19,4)
Declare @Othrscnt int
Declare @DOThrs Decimal(19,4)
Declare @ArrearsDOThrs Decimal(19,4)
Declare @DOthrscnt int
Declare @TOThrs Decimal(19,4)
Declare @TOthrscnt int
Declare @Latehrs Decimal(19,4)
Declare @Latehrscnt int
Declare @dayoffs Decimal(19,4)
Declare @dayoffscnt int
Declare @daysworked Decimal(19,4)
Declare @daysworkedcnt int
Declare @AppLeave Decimal(19,4)
Declare @AppLeavecnt int
Declare @NAppLeave Decimal(19,4)
Declare @NAppLeavecnt int
Declare @Poya Decimal(19,4)
Declare @Poyacnt int
Declare @Mercantile Decimal(19,4)
Declare @Mercantilecnt int
Declare @sunday Decimal(19,5)
Declare @sundaycnt int
Declare @AutoNopayDays int
declare @EmpNoPaydevider int
Declare @TotSundays int
Declare @Date date
Declare @Deducted decimal(18,2)
set @Days =0
set @cnt =0
Declare @auonopaycount int
--declare @Employeecode int
declare @x int
declare @StartDate int
declare @EndtDate int
--declare @SchemeCode int
declare @autonopay decimal(18,2)
declare @ThisMonth int
declare @ThisYear int
declare @LastMonth int
declare @LastYear int
declare @Location int
Declare @Inlate decimal(18,2)
Declare @OutLate decimal(18,2)
Declare @Balance decimal(18,2)
Declare @DayType int
Declare @DOA date
Declare @LeaveYearStartDate date
Declare @LeaveYearEndDate date
Declare @LeaveYear int
Declare @LeaveType int
Declare @Val decimal(18,2)
Declare @LastMonthStartDate date
Declare @LastMonthEndDate date
Declare @DOB date
Declare @DOR date
Declare @LastMonthGrossSalary decimal(18,2)
Declare @ArrearDays int
Declare @daysArrearscnt int
Declare @AnnualLeaveType int=1
Declare @CasualLeaveType int=2
Declare @SickLeaveType int=3
Declare @PensionScheme int
Declare @OTAmount Decimal(18,2)
DECLARE @Success bit
DECLARE @Message varchar(50)
Declare @transcount int
Declare @Count as decimal(18,10)
Declare @Empcode as int
Declare @Progress as decimal(18,2)
select @Count=COUNT([dbo].[MI_Employer_Employee].EmployeeCode) from [dbo].[MI_PROLL_ProcessEmployee] inner join [dbo].[MI_Employer_Employee] on
[dbo].[MI_PROLL_ProcessEmployee].employeecode=[dbo].[MI_Employer_Employee].employeecode where [dbo].[MI_Employer_Employee].deleted=0
and [dbo].[MI_Employer_Employee].schemecode not in (@SchemeCode) and invokedby=@InvokedBy and [dbo].[MI_PROLL_ProcessEmployee].deleted=0 --and [dbo].[MI_Employer_Employee].EmployeeCode=382
IF @Count=0 
BEGIN
	SET @Count=1
END
SET @Count=(1/@Count)*100
--select @Count
update [dbo].[MI_PROLL_PayrollProcessing] set Progress=0,Updatedby=@InvokedBy,Updateddate=getDate() where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=0;
DECLARE Input_all cursor for
--Select [dbo].[MI_Employer_Employee].EmployeeCode from [dbo].[MI_PROLL_ProcessEmployee] inner join [dbo].[MI_Employer_Employee] on
--[dbo].[MI_PROLL_ProcessEmployee].employeecode=[dbo].[MI_Employer_Employee].employeecode where [dbo].[MI_Employer_Employee].deleted=0
--and [dbo].[MI_Employer_Employee].schemecode not in (@) and invokedby=@InvokedBy 
--and [dbo].[MI_PROLL_ProcessEmployee].deleted=0 --and [dbo].[MI_Employer_Employee].EmployeeCode=137
SELECT e.EmployeeCode
		FROM [dbo].[MI_PROLL_ProcessEmployee] AS p
			 INNER JOIN
			 [dbo].[mi_employer_employee] AS e
			 ON p.employeecode = e.employeecode
		WHERE e.Deleted = 0 AND 
			  e.schemecode IN( @SchemeCode ) AND 
			  invokedby = @InvokedBy AND 
		  p.Deleted = 0 
--and e.EmployeeCode in (382)
--and    e.Level4Code in (
--2
--,3
--,4
--,5
--,6
--,7
--,8
--,9
--,10
--,11
--,12
--,13
--,14
--,17
--,18
--,19
--,20
--,23
--,25
--,30
--,33
--,34
--,35
--,38
--,42
--,44
--,45
--,46
--,47
--,50
--,54
--,56
--,62
--,63
--,64
--,67
--,68
--,73
--,83
--,84
--,89
--,90
--,92
--,101
--) 
and e.Deleted=0
--and e.Level3Code=10 and e.Level4Code=33 
order by e.EmployeeCode
OPEN Input_all
FETCH NEXT FROM Input_all into @Empcode
while @@FETCH_STATUS=0 
BEGIN

insert into [my_log] values (concat('@Empcode',@Empcode), getdate())
--While(@Progress<100)
--BEGIN 
	
	SET @Progress =(SELECT (Progress+@Count) FROM [dbo].[MI_PROLL_PayrollProcessing] where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=0);
	set @Progress=round(@Progress,2)
	if @Progress>=100 begin
		set @Progress=100
	end
	update [dbo].[MI_PROLL_PayrollProcessing] set Progress=@Progress,Updatedby=@InvokedBy,Updateddate=getDate() where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=0;
	--WAITFOR DELAY '00:00:01'
--END--Change
--select @Progress


select @DOA=DateOfAppointment from [dbo].MI_Employer_Employee where employeecode=@Empcode and Deleted=0
select @PensionScheme=PensionSchemeCode  from [dbo].MI_Employer_Employee where employeecode=@Empcode
set @PensionScheme=isnull(@PensionScheme,0)
--SELECT @LastMonthGrossSalary=sum(ProcessValue) FROM [dbo].MI_pRoll_ProcessInfo where Description ='Gross Salary' and EmployeeCode=@Empcode and PayPeriodCode=@PayPeriodCode-1
--set @LastMonthGrossSalary= isnull(@LastMonthGrossSalary,0)
select @SchemeCode=SchemeCode, @Location=LocationCode, @DOA=DateOfAppointment, @DOR=ResignedDate from [dbo].MI_Employer_Employee where EmployeeCode=@Empcode and Deleted=0
--select cast(SchemeItemValue as int) from [dbo].MI_pRoll_SchemeDetails where SchemeItemCode=15 and SchemeCode=8
--select cast(SchemeItemValue as int) from [dbo].MI_pRoll_SchemeDetails where SchemeItemCode=16 and SchemeCode=8
select month(StartDate) from [dbo].MI_pRoll_PayPeriod where PayPeriodCode=@PayPeriodCode  and PPeriodTypeCode=3
select @ThisYear=Year(StartDate) from [dbo].MI_pRoll_PayPeriod where PayPeriodCode=@PayPeriodCode and PPeriodTypeCode=3
select @ThisMonth=Month(StartDate) from [dbo].MI_pRoll_PayPeriod where PayPeriodCode=@PayPeriodCode and PPeriodTypeCode=3
select @StartDate=Day(StartDate) from [dbo].MI_pRoll_PayPeriod where PayPeriodCode=@PayPeriodCode and PPeriodTypeCode=3
select @EndtDate=Day(EndDate) from [dbo].MI_pRoll_PayPeriod where PayPeriodCode=@PayPeriodCode and PPeriodTypeCode=3

--if @ThisMonth=1 begin
--	set @LastYear=@ThisYear-1
--	set @LastMonth=12
--end
--else begin
--	set @LastMonth=@ThisMonth-1
--	set @LastYear=@ThisYear
--end

if @ThisMonth=1 begin
	if @SchemeCode>1 begin
		set @LastYear=@ThisYear
		set @LastMonth=1
	end
	else begin
		set @LastYear=@ThisYear
		set @LastMonth=1
	end
end
else begin
	if @SchemeCode>1 begin
		set @LastMonth=@ThisMonth
		set @LastYear=@ThisYear
	end
	else begin
		set @LastMonth=@ThisMonth-1
		set @LastYear=@ThisYear
	end
end

set @SDate=cast(cast(@LastYear as varchar(max))+'-'+cast(@LastMonth as varchar(max))+'-'+cast(@StartDate as varchar(max)) as date)
set @EDate=cast(cast(@ThisYear as varchar(max))+'-'+cast(@ThisMonth as varchar(max))+'-'+cast(@EndtDate as varchar(max)) as date)

select @LSDate=Cast(StartDate as Date) from [dbo].mi_proll_payperiod where payperiodCode=(select payperiodCode-1 from [dbo].mi_proll_payperiod where cast(StartDate as Date)=cast(@SDate as Date))
select @LEDate=Cast(EndDate as Date) from [dbo].mi_proll_payperiod where payperiodCode=(select payperiodCode-1 from [dbo].mi_proll_payperiod where cast(enddate as Date)=cast(@EDate as Date))

--set @LSDate=DATEADD(M,-1,@SDate)
--set @LEDate=DATEADD(M,-1,@EDate)
/*set @LeaveYearStartDate=cast(cast(@ThisYear as varchar(10))+'-'+Cast(Month(@DOA)as varchar(10))+'-'+Cast(day(@DOA)as varchar(10)) as Date)

if Month(@LeaveYearStartDate)<Month(@EDate) begin
	set @LeaveYearStartDate=cast(cast(@ThisYear as varchar(10))+'-'+Cast(Month(@DOA)as varchar(10))+'-'+Cast(day(@DOA)as varchar(10)) as Date)
	if (Day(@DOA)=1 and Month(@DOA)=1) begin
		set @LeaveYearEndDate=cast(Cast(@ThisYear as varchar(10))+'-'+Cast(Month(DateAdd(D,-1,@DOA)) as varchar(10))+'-'+Cast(day(DateAdd(D,-1,@DOA)) as varchar(10)) as Date)
	end
	else begin
		--set @LeaveYearEndDate=cast(Cast(@ThisYear+1 as varchar(10))+'-'+Cast(Month(DateAdd(D,-1,@DOA)) as varchar(10))+'-'+Cast(day(DateAdd(D,-1,@DOA)) as varchar(10)) as Date)
		set @LeaveYearEndDate=cast(Cast(@ThisYear+1 as varchar(10))+'-'+Cast(Month(@DOA) as varchar(10))+'-'+Cast(day(@DOA) as varchar(10)) as Date)
		set @LeaveYearEndDate=DateAdd(D,-1,@LeaveYearEndDate)
	end
end
else begin
	set @LeaveYearStartDate=DateAdd(D,-1,DATEADD(YYYY,1,@DOA))
	set @LeaveYearStartDate=Convert(datetime,cast(@ThisYear-1 as varchar(10))+'-'+Cast(Month(@DOA)as varchar(10))+'-'+Cast(day(@DOA)as varchar(10)))

	set @LeaveYearEndDate=DateAdd(YYYY,1,@LeaveYearStartDate)
	set @LeaveYearEndDate=DateAdd(day,-1,@LeaveYearEndDate)
end

if @EDate<@LeaveYearStartDate begin
	Set @LeaveYear=Year(@LeaveYearStartDate)-1
end
else if @EDate<=@LeaveYearEndDate and @EDate>=@LeaveYearStartDate begin
	Set @LeaveYear=Year(@LeaveYearStartDate)
end
else if @EDate>@LeaveYearEndDate begin
	Set @LeaveYear=Year(@LeaveYearStartDate)+1
end
*/

Set @LeaveYear= (
					SELECT LeaveYear
					FROM [dbo].MI_ONTIME_LeavePeriod
					WHERE IsDefault = 1
					)
BEGIN TRY	

--========================Auto Nopay days================================================
			--if @Empcode not in (Select employeecode from [dbo].mi_employer_employee where number in(Select number from [dbo].MI_Attendance_ExemptedEmployees)) begin
			--	--Late employees
			--	Declare @Annual decimal(18,2)
			--	set @Val=0
			--	set @Cnt=0
			--	select @Annual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode  and Payperiod=@PayPeriodCode and LeaveType=@AnnualLeaveType
			--	set @Annual=isnull(@Annual,0)
			--	select @Cnt=count(EmployeeCode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--	if @Cnt>0 begin
			--		update [dbo].MI_ONTIME_LeaveAllocation set Used=Used-@Annual where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--		select  @Annual=used from [dbo].MI_ONTIME_LeaveAllocation  where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--		if @Annual<0 begin
			--			update [dbo].MI_ONTIME_LeaveAllocation set Used=0 where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--		end
			--	end

			--	Declare @Casual decimal(18,2)
			--	set @Cnt=0
			--	select @Casual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode  and Payperiod=@PayPeriodCode and LeaveType=@CasualLeaveType
			--	set @Casual=isnull(@Casual,0)
			--	select @Cnt=count(EmployeeCode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--	if @Cnt>0 begin
			--		update [dbo].MI_ONTIME_LeaveAllocation set Used=Used-@Casual where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--		select  @Casual=used from [dbo].MI_ONTIME_LeaveAllocation  where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--		if @Casual<0 begin
			--			update [dbo].MI_ONTIME_LeaveAllocation set Used=0 where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--		end
			--	end
				
				
			--	Declare @Sick decimal(18,2)
			--	set @Cnt=0
			--	select @Sick=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode  and Payperiod=@PayPeriodCode and LeaveType=@SickLeaveType
			--	set @Sick=isnull(@Sick,0)
			--	select @Cnt=count(EmployeeCode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--	if @Cnt>0 begin
			--		update [dbo].MI_ONTIME_LeaveAllocation set Used=Used-@Sick where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--		select  @Sick=used from [dbo].MI_ONTIME_LeaveAllocation  where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--		if @Sick<0 begin
			--			update [dbo].MI_ONTIME_LeaveAllocation set Used=0 where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--		end
			--	end
			--	insert into my_log values(concat('@Sdate ',@Sdate, '@Edate ',@Edate, '@EmpCode ',@EmpCode), getdate())
					
			--	Delete from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode 
			--	--if @SchemeCode=12 or (@SchemeCode=21) begin
			--		if (@ThisMonth=month(@DOR) and @ThisYear=Year(@DOR)) and @LastMonthGrossSalary=0 begin
			--			declare EffectedEmployee cursor for select Date, Inlate,OutLate,DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @LSdate and date <= @DOR and InLate>0
			--		end
			--		else if (@ThisMonth=month(@DOR) and @ThisYear=Year(@DOR)) and @LastMonthGrossSalary>0 begin
			--			declare EffectedEmployee cursor for select Date, Inlate,OutLate,DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @Sdate and date <= @DOR and inlate>0
			--		end
			--		else if (@LastMonth=month(@DOA) and @LastYear=Year(@DOA)) and @LastMonthGrossSalary=0 begin
			--			declare EffectedEmployee cursor for select Date, Inlate,OutLate,DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @LSdate and date <= @Edate and inlate>0
			--		end
			--		else begin
			--			declare EffectedEmployee cursor for select Date, Inlate,OutLate, DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @Sdate and date <= @Edate and inlate>0
			--		end
					
			--		open EffectedEmployee
			--		fetch next from EffectedEmployee into @Date,@Inlate,@OutLate,@DayType
			--		while @@Fetch_status = 0
			--		begin
							
			--			if @Inlate>0 or @Inlate>0 begin 
			--				set @Val=@Val+1
			--			end
							
			--		fetch next from EffectedEmployee into @Date,@Inlate,@OutLate,@DayType
			--		End
			--		close EffectedEmployee
			--		deallocate EffectedEmployee
			--		Declare @AnnualBalance decimal(18,2)
			--		Declare @CasualBalance decimal(18,2)
			--		Declare @SickBalance decimal(18,2)
			--		Declare @AnnualPending decimal(18,2)
			--		Declare @CasualPending decimal(18,2)
			--		Declare @SickPending decimal(18,2)
			--		Declare @Remain decimal(18,2)
			--						insert into my_log values(concat('@@Val ',@Val), getdate())

			--		set @Val=@Val/6
			--		if @Val>0 begin
			--			set @Deducted=0
			--			select @Deducted=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Payperiod=@PayPeriodCode and Employeecode=@Empcode and @LeaveType=@AnnualLeaveType
			--			set @Deducted=isnull(@Deducted,0)
			--			set @cnt=0
			--			select @cnt=count(employeecode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--			if @cnt>0 begin
			--				--select @AnnualPending=[Z1_Multinet].[Fn_GetPendingLeaveCount] (@Empcode, @LeaveYear, @AnnualLeaveType)
			--				Select @AnnualPending= NoOfDays FROM [dbo].MI_ONTIME_Leave WHERE Deleted = 0 AND EmployeeCode = @EmpCode AND LeaveYear = @LeaveYear AND LeaveTypeCode = @LeaveType AND RequestStatus = 'P'
			--				set @AnnualPending=isnull(@AnnualPending,0)
			--				select @AnnualBalance=Allocated-(@AnnualPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--				set @AnnualBalance=isnull(@AnnualBalance,0)-@Deducted
			--			end
			--			set @cnt=0
			--			set @Deducted=0
			--			select @Deducted=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Payperiod=@PayPeriodCode and Employeecode=@Empcode and @LeaveType=@CasualLeaveType
			--			set @Deducted=isnull(@Deducted,0)
			--			set @cnt=0
			--			select @cnt=count(employeecode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--			if @cnt>0 begin
			--				select @CasualPending=NoOfDays FROM [dbo].MI_ONTIME_Leave WHERE Deleted = 0 AND EmployeeCode = @EmpCode AND LeaveYear = @LeaveYear AND LeaveTypeCode = @LeaveType AND RequestStatus = 'P'
			--				set @CasualPending=isnull(@CasualPending,0)
			--				select @CasualBalance=Allocated-(@CasualPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--				set @CasualBalance=isnull(@CasualBalance,0)-@Deducted
			--			end
			--			set @cnt=0
			--			set @Deducted=0
			--			select @Deducted=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Payperiod=@PayPeriodCode and Employeecode=@Empcode and @LeaveType=@SickLeaveType
			--			set @Deducted=isnull(@Deducted,0)
			--			set @cnt=0
			--			select @cnt=count(employeecode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--			if @cnt>0 begin
			--				select @SickPending=NoOfDays FROM [dbo].MI_ONTIME_Leave WHERE Deleted = 0 AND EmployeeCode = @EmpCode AND LeaveYear = @LeaveYear AND LeaveTypeCode = @LeaveType AND RequestStatus = 'P'
			--				set @SickPending=isnull(@SickPending,0)
			--				select @SickBalance=Allocated-(@SickPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--				set @SickBalance=isnull(@SickBalance,0)-@Deducted
			--			end	



			--		if @AnnualBalance>@Val begin
			--						Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@AnnualLeaveType,@Val)
			--					end
			--					else begin
			--						if @AnnualBalance>0 begin
			--							Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@AnnualLeaveType,@AnnualBalance)
			--							set @Remain=@Val-@AnnualBalance
			--							if @Remain>0 begin
			--								if @CasualBalance>@Remain begin
			--									Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@CasualLeaveType,@Remain)
			--								end
			--								else begin
			--									Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@CasualLeaveType,@CasualBalance)
			--									set @Remain=@Val-@CasualBalance
			--									if @Remain>0 begin
			--										if @SickBalance>@Remain begin
			--											Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@SickLeaveType,@Remain)
			--										end
			--										else begin
			--											Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@SickLeaveType,@SickBalance)
			--											set @Remain=@Val-@SickBalance
			--											if @Remain>0 begin
			--												Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',0,@Remain)
			--											end



			--			--if @CasualBalance>@Val begin
			--			--	Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@CasualLeaveType,@Val, getdate())
			--			--end
			--			--else begin
			--			--	if @CasualBalance>0 begin
			--			--		Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@CasualLeaveType,@CasualBalance,getdate())
			--			--		set @Remain=@Val-@CasualBalance
			--			--		if @Remain>0 begin
			--			--			if @SickBalance>@Remain begin
			--			--				Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@SickLeaveType,@Remain,getdate())
			--			--			end
			--			--			else begin
			--			--				Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@SickLeaveType,@SickBalance,getdate())
			--			--				set @Remain=@Val-@SickBalance
			--			--				if @Remain>0 begin
			--			--					if @AnnualBalance>@Remain begin
			--			--						Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@AnnualLeaveType,@Remain,getdate())
			--			--					end
			--			--					else begin
			--			--						Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',@AnnualLeaveType,@AnnualBalance,getdate())
			--			--						set @Remain=@Val-@AnnualBalance
			--			--						if @Remain>0 begin
			--			--							Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'3 Days Late',0,@Remain,getdate())
			--			--						end
			--								end
			--							end
			--						end
			--					end
			--				end
			--			end

							
			--			delete [dbo].MI_EPRF_AutoLeaveDeduction where Employeecode=@Empcode and Payperiod=@PayPeriodCode and Days=0
			--		end
			--		set @cnt=0
			--		Declare @used decimal(18,2)
			--		select @Annual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=@AnnualLeaveType
			--		set @Annual=isnull(@Annual,0)
			--		if @Annual>0 begin
			--			select @used=used from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--			set @used=isnull(@used,0)
			--			set @used=@used+@Annual
			--			update [dbo].MI_ONTIME_LeaveAllocation set Used=@Used where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
			--		end
			--		select @Casual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=@CasualLeaveType
			--		set @Casual=isnull(@Casual,0)
			--		if @Casual>0 begin
			--			select @used=used from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--			set @used=isnull(@used,0)
			--			set @used=@used+@Casual
			--			update [dbo].MI_ONTIME_LeaveAllocation set Used=@Used where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
			--		end
			--		select @Sick=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=@SickLeaveType
			--		set @Sick=isnull(@Sick,0)
			--		if @Sick>0 begin
			--			select @used=used from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--			set @used=isnull(@used,0)
			--			set @used=@used+@Sick
			--			update [dbo].MI_ONTIME_LeaveAllocation set Used=@Used where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
			--		end
						
			--	--end
			--end
			--if @Empcode not in (Select employeecode from [dbo].MI_Employer_Employee where number in(Select number from [dbo].MI_Attendance_ExemptedEmployees)) begin
					
					
			--	if (@ThisMonth=month(@DOR) and @ThisYear=Year(@DOR)) and @LastMonthGrossSalary=0 begin
			--		declare EffectedEmployee cursor for
			--		select Date from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @LSdate and date <= @DOR and Present=0 and Expected=1 and LeaveAmount=0
			--	end
			--	else begin
			--		declare EffectedEmployee cursor for
			--		select Date from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @Sdate and date <= @Edate and Present=0 and Expected=1 and LeaveAmount=0
			--	end
					
			--	open EffectedEmployee
			--	fetch next from EffectedEmployee into @Date
			--	while @@Fetch_status = 0
			--		begin		

			--			DECLARE @annualBalanceTemp Decimal(18,2) = 0;	--1
			--			DECLARE @casualBalanceTemp Decimal(18,2) = 0;	--2
			--			DECLARE @sickBalanceTemp Decimal(18,2) = 0;		--3

			--			SELECT @annualBalanceTemp = (Allocated-Used) from MI_ONTIME_LeaveAllocation where EmployeeCode = @Empcode and LeaveTypeCode = 1 and LeaveYear = @LeaveYear
			--			SELECT @casualBalanceTemp = (Allocated-Used) from MI_ONTIME_LeaveAllocation where EmployeeCode = @Empcode and LeaveTypeCode = 2 and LeaveYear = @LeaveYear
			--			SELECT @sickBalanceTemp = (Allocated-Used) from MI_ONTIME_LeaveAllocation where EmployeeCode = @Empcode and LeaveTypeCode = 3 and LeaveYear = @LeaveYear


			--			IF @sickBalanceTemp > 1
			--				SET @LeaveType = 3;
			--			ELSE IF @casualBalanceTemp > 1
			--				SET @LeaveType = 2;
			--			ELSE IF @annualBalanceTemp > 1
			--				SET @LeaveType = 1;
			--			ELSE
			--				SET @LeaveType = 0;
			--			Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@Date,0,'Absent',0,1)		
			--	fetch next from EffectedEmployee into @Date
			--	End
			--	close EffectedEmployee
			--	deallocate EffectedEmployee
			--	set @Autonopay=isnull(@Autonopay,0)
			--	select @Autonopay=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Employeecode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=0
			--	set @Autonopay=isnull(@Autonopay,0)
			--end

			if @Empcode not in (Select employeecode from [dbo].mi_employer_employee where number in(Select number from [dbo].MI_Attendance_ExemptedEmployees)) begin
				--Late employees
				Declare @Annual decimal(18,2)
				set @Val=0
				set @Cnt=0
				select @Annual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode  and Payperiod=@PayPeriodCode and LeaveType=@AnnualLeaveType
				set @Annual=isnull(@Annual,0)
				select @Cnt=count(EmployeeCode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
				if @Cnt>0 begin
					update [dbo].MI_ONTIME_LeaveAllocation set Used=Used-@Annual where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
					select  @Annual=used from [dbo].MI_ONTIME_LeaveAllocation  where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
					if @Annual<0 begin
						update [dbo].MI_ONTIME_LeaveAllocation set Used=0 where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
					end
				end

				Declare @Casual decimal(18,2)
				set @Cnt=0
				select @Casual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode  and Payperiod=@PayPeriodCode and LeaveType=@CasualLeaveType
				set @Casual=isnull(@Casual,0)
				select @Cnt=count(EmployeeCode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
				if @Cnt>0 begin
					update [dbo].MI_ONTIME_LeaveAllocation set Used=Used-@Casual where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
					select  @Casual=used from [dbo].MI_ONTIME_LeaveAllocation  where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
					if @Casual<0 begin
						update [dbo].MI_ONTIME_LeaveAllocation set Used=0 where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
					end
				end
				
				
				Declare @Sick decimal(18,2)
				set @Cnt=0
				select @Sick=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode  and Payperiod=@PayPeriodCode and LeaveType=@SickLeaveType
				set @Sick=isnull(@Sick,0)
				select @Cnt=count(EmployeeCode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
				if @Cnt>0 begin
					update [dbo].MI_ONTIME_LeaveAllocation set Used=Used-@Sick where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
					select  @Sick=used from [dbo].MI_ONTIME_LeaveAllocation  where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
					if @Sick<0 begin
						update [dbo].MI_ONTIME_LeaveAllocation set Used=0 where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
					end
				end

				insert into [my_log] values (concat('@@Sick',@Sick,'@Casual',@Casual,'@Annual',@Annual), getdate())

				Delete from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode 
				--if @SchemeCode=12 or (@SchemeCode=21) begin
					if (@ThisMonth=month(@DOR) and @ThisYear=Year(@DOR)) and @LastMonthGrossSalary=0 begin
						declare EffectedEmployee cursor for select Date, Inlate,OutLate,DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @LSdate and date <= @DOR and InLate>0
					end
					else if (@ThisMonth=month(@DOR) and @ThisYear=Year(@DOR)) and @LastMonthGrossSalary>0 begin
						declare EffectedEmployee cursor for select Date, Inlate,OutLate,DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @Sdate and date <= @DOR and inlate>0
					end
					else if (@LastMonth=month(@DOA) and @LastYear=Year(@DOA)) and @LastMonthGrossSalary=0 begin
						declare EffectedEmployee cursor for select Date, Inlate,OutLate,DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @LSdate and date <= @Edate and inlate>0
					end
					else begin
						declare EffectedEmployee cursor for select Date, Inlate,OutLate, DayTypeCode from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @Sdate and date <= @Edate and inlate>0
					end
					open EffectedEmployee
					fetch next from EffectedEmployee into @Date,@Inlate,@OutLate,@DayType
					while @@Fetch_status = 0
					begin
							
						if @Inlate>0 or @Inlate>0 begin 
							set @Val=@Val+1
						end
							
					fetch next from EffectedEmployee into @Date,@Inlate,@OutLate,@DayType
					End
					close EffectedEmployee
					deallocate EffectedEmployee
					Declare @AnnualBalance decimal(18,2)
					Declare @CasualBalance decimal(18,2)
					Declare @SickBalance decimal(18,2)
					Declare @AnnualPending decimal(18,2)
					Declare @CasualPending decimal(18,2)
					Declare @SickPending decimal(18,2)
					Declare @Remain decimal(18,2)
					insert into [my_log] values (concat('@Val',@Val), getdate())
					
					set @Val=@Val/3
					set @Val=cast(@Val as int)
					set @Val= @Val/2

					if @Val>0 begin
					insert into [my_log] values (concat('@@Val>0 ','=true'), getdate())

						set @Deducted=0
						select @Deducted=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Payperiod=@PayPeriodCode and Employeecode=@Empcode and @LeaveType=@AnnualLeaveType
						set @Deducted=isnull(@Deducted,0)
						set @cnt=0
						select @cnt=count(employeecode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
						if @cnt>0 begin
							--select @AnnualPending=[Z1_Multinet].[Fn_GetPendingLeaveCount] (@Empcode, @LeaveYear, @AnnualLeaveType)
							Select @AnnualPending= NoOfDays FROM [dbo].MI_ONTIME_Leave WHERE Deleted = 0 AND EmployeeCode = @EmpCode AND LeaveYear = @LeaveYear AND LeaveTypeCode = @LeaveType AND RequestStatus = 'P'
							set @AnnualPending=isnull(@AnnualPending,0)
							select @AnnualBalance=Allocated-(@AnnualPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
							set @AnnualBalance=isnull(@AnnualBalance,0)-@Deducted
						end
						set @cnt=0
						set @Deducted=0
						select @Deducted=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Payperiod=@PayPeriodCode and Employeecode=@Empcode and @LeaveType=@CasualLeaveType
						set @Deducted=isnull(@Deducted,0)
						set @cnt=0
						select @cnt=count(employeecode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
						if @cnt>0 begin
							select @CasualPending=NoOfDays FROM [dbo].MI_ONTIME_Leave WHERE Deleted = 0 AND EmployeeCode = @EmpCode AND LeaveYear = @LeaveYear AND LeaveTypeCode = @LeaveType AND RequestStatus = 'P'
							set @CasualPending=isnull(@CasualPending,0)
							select @CasualBalance=Allocated-(@CasualPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
							set @CasualBalance=isnull(@CasualBalance,0)-@Deducted
						end
						set @cnt=0
						set @Deducted=0
						select @Deducted=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Payperiod=@PayPeriodCode and Employeecode=@Empcode and @LeaveType=@SickLeaveType
						set @Deducted=isnull(@Deducted,0)
						set @cnt=0
						select @cnt=count(employeecode) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
						insert into [my_log] values (concat('@@@cnt',@cnt, '@@@LeaveYear',@LeaveYear,'@SickLeaveType',@SickLeaveType), getdate())
						if @cnt>0 begin
							select @SickPending=NoOfDays FROM [dbo].MI_ONTIME_Leave WHERE Deleted = 0 AND EmployeeCode = @EmpCode AND LeaveYear = @LeaveYear AND LeaveTypeCode = @LeaveType AND RequestStatus = 'P'
							set @SickPending=isnull(@SickPending,0)
							select @SickBalance=Allocated-(@SickPending+Used) from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
							insert into [my_log] values (concat('@@SickBalance',@SickBalance, '@@Deducted',@Deducted), getdate())

							set @SickBalance=isnull(@SickBalance,0)-@Deducted
						end	


					insert into [my_log] values (concat('@Val',@Val, '@SickBalance',@SickBalance), getdate())
					if @SickBalance>@Val begin
									Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late from Sick',@SickLeaveType,@Val)
														insert into [my_log] values (concat('@Val',@Val,'@Empcode',@Empcode, '@PayPeriodCode',@PayPeriodCode), getdate())

								end
								else begin
									insert into [my_log] values (concat('@Val',@Val, '@SickBalance',@SickBalance), getdate())
									if @SickBalance>0 begin
										Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late from Sick',@SickLeaveType,@SickBalance)
										set @Remain=@Val-@SickBalance
										if @Remain>0 begin
											if @CasualBalance>@Remain begin
												Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late from Casual',@CasualLeaveType,@Remain)
											end
											else begin
												Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late from Casual',@CasualLeaveType,@CasualBalance)
												set @Remain=@Val-@CasualBalance
												if @Remain>0 begin
													if @AnnualBalance>@Remain begin
														Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late from Annual',@AnnualLeaveType,@Remain)
													end
													else begin
														Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late from Annual',@AnnualLeaveType,@AnnualBalance)
														set @Remain=@Val-@AnnualBalance

														if @Remain>0 begin
															Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Leave Balance Exhausted',0,@Remain)
														end

					--if @AnnualBalance>@Val begin
					--				Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@AnnualLeaveType,@Val)
					--			end
					--			else begin
					--				if @AnnualBalance>0 begin
					--					Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@AnnualLeaveType,@AnnualBalance)
					--					set @Remain=@Val-@AnnualBalance
					--					if @Remain>0 begin
					--						if @CasualBalance>@Remain begin
					--							Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@CasualLeaveType,@Remain)
					--						end
					--						else begin
					--							Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@CasualLeaveType,@CasualBalance)
					--							set @Remain=@Val-@CasualBalance
					--							if @Remain>0 begin
					--								if @SickBalance>@Remain begin
					--									Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@SickLeaveType,@Remain)
					--								end
					--								else begin
					--									Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@SickLeaveType,@SickBalance)
					--									set @Remain=@Val-@SickBalance

					--									if @Remain>0 begin
					--										Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',0,@Remain)
					--									end



						--if @CasualBalance>@Val begin
						--	Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@CasualLeaveType,@Val, getdate())
						--end
						--else begin
						--	if @CasualBalance>0 begin
						--		Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@CasualLeaveType,@CasualBalance,getdate())
						--		set @Remain=@Val-@CasualBalance
						--		if @Remain>0 begin
						--			if @SickBalance>@Remain begin
						--				Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@SickLeaveType,@Remain,getdate())
						--			end
						--			else begin
						--				Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@SickLeaveType,@SickBalance,getdate())
						--				set @Remain=@Val-@SickBalance
						--				if @Remain>0 begin
						--					if @AnnualBalance>@Remain begin
						--						Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@AnnualLeaveType,@Remain,getdate())
						--					end
						--					else begin
						--						Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',@AnnualLeaveType,@AnnualBalance,getdate())
						--						set @Remain=@Val-@AnnualBalance
						--						if @Remain>0 begin
						--							Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days,UpdatedOn) values (@Empcode,@PayPeriodCode,@SDate,0,'Half Day On 3 Days Late',0,@Remain,getdate())
						--						end
											end
										end
									end
								end
							end
						end

							
						delete [dbo].MI_EPRF_AutoLeaveDeduction where Employeecode=@Empcode and Payperiod=@PayPeriodCode and Days=0
					end
					set @cnt=0
					Declare @used decimal(18,2)
					select @Annual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=@AnnualLeaveType
					set @Annual=isnull(@Annual,0)
					if @Annual>0 begin
						select @used=used from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
						set @used=isnull(@used,0)
						set @used=@used+@Annual
						update [dbo].MI_ONTIME_LeaveAllocation set Used=@Used where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@AnnualLeaveType
					end
					select @Casual=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=@CasualLeaveType
					set @Casual=isnull(@Casual,0)
					if @Casual>0 begin
						select @used=used from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
						set @used=isnull(@used,0)
						set @used=@used+@Casual
						update [dbo].MI_ONTIME_LeaveAllocation set Used=@Used where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@CasualLeaveType
					end
					select @Sick=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where EmployeeCode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=@SickLeaveType
					set @Sick=isnull(@Sick,0)
					if @Sick>0 begin
						select @used=used from [dbo].MI_ONTIME_LeaveAllocation where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
						set @used=isnull(@used,0)
						set @used=@used+@Sick
						update [dbo].MI_ONTIME_LeaveAllocation set Used=@Used where EmployeeCode=@Empcode and LeaveYear=@LeaveYear  and LeaveTypeCode=@SickLeaveType
					end
						
				--end
			end
			if @Empcode not in (Select employeecode from [dbo].MI_Employer_Employee where number in(Select number from [dbo].MI_Attendance_ExemptedEmployees)) begin
					
					
				if (@ThisMonth=month(@DOR) and @ThisYear=Year(@DOR)) and @LastMonthGrossSalary=0 begin
					declare EffectedEmployee cursor for
					select Date from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @LSdate and date <= @DOR and Present=0 and Expected=1 and LeaveAmount=0
				end
				else begin
					declare EffectedEmployee cursor for
					select Date from [dbo].MI_ONTIME_Attendance where EmployeeCode =@EmpCode and date >= @Sdate and date <= @Edate and Present=0 and Expected=1 and LeaveAmount=0
				end
					
				open EffectedEmployee
				fetch next from EffectedEmployee into @Date
				while @@Fetch_status = 0
					begin		

						DECLARE @annualBalanceTemp Decimal(18,2) = 0;	--1
						DECLARE @casualBalanceTemp Decimal(18,2) = 0;	--2
						DECLARE @sickBalanceTemp Decimal(18,2) = 0;		--3

						SELECT @annualBalanceTemp = (Allocated-Used) from MI_ONTIME_LeaveAllocation where EmployeeCode = @Empcode and LeaveTypeCode = 1 and LeaveYear = @LeaveYear
						SELECT @casualBalanceTemp = (Allocated-Used) from MI_ONTIME_LeaveAllocation where EmployeeCode = @Empcode and LeaveTypeCode = 2 and LeaveYear = @LeaveYear
						SELECT @sickBalanceTemp = (Allocated-Used) from MI_ONTIME_LeaveAllocation where EmployeeCode = @Empcode and LeaveTypeCode = 3 and LeaveYear = @LeaveYear


						IF @sickBalanceTemp > 1
							SET @LeaveType = 3;
						ELSE IF @casualBalanceTemp > 1
							SET @LeaveType = 2;
						ELSE IF @annualBalanceTemp > 1
							SET @LeaveType = 1;
						ELSE
							SET @LeaveType = 0;
						Insert into [dbo].MI_EPRF_AutoLeaveDeduction (Employeecode, Payperiod, Date, LateHours, Reason,LeaveType,Days) values (@Empcode,@PayPeriodCode,@Date,0,'Absent',0,1)		
				fetch next from EffectedEmployee into @Date
				End
				close EffectedEmployee
				deallocate EffectedEmployee
				set @Autonopay=isnull(@Autonopay,0)
				select @Autonopay=sum(Days) from [dbo].MI_EPRF_AutoLeaveDeduction where Employeecode=@Empcode and Payperiod=@PayPeriodCode and LeaveType=0
				set @Autonopay=isnull(@Autonopay,0)
			end

--========================End Auto Nopay days================================================
			
			END TRY
				BEGIN CATCH
							SELECT ERROR_MESSAGE() as ErrorMessage
							SELECT GETDATE()
							--insert into [dbo].error values (ERROR_MESSAGE(),GETDATE(),@EmpCode)
							set @Success = 0
							set @Message = 'Error ocured while syncing'
							
				END CATCH;
	FETCH NEXT FROM Input_all into @Empcode
	
END
close Input_all
Deallocate Input_all
--Update [dbo].[MI_PROLL_ProcessEmployee] SET Deleted=1
--update [dbo].[MI_PROLL_PayrollProcessing] set Deleted=1 where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=0
--update [dbo].[MI_PROLL_PayrollProcessing] set ProcessingStatus=1 where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=1
UPDATE [dbo].[MI_PROLL_ProcessEmployee]
	  SET Deleted = 1;
update [dbo].[MI_PROLL_PayrollProcessing] set Processingstatus=1 where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=0
	--update [dbo].[MI_PROLL_PayrollProcessing] set Deleted=1 where TemplateCode=@TemplateCode AND InvokedBy=@InvokedBy AND Deleted=0
	--select @Message As Message, @Success As Success

				--select @Message As Message, @Success As Success


IF @@ERROR <> 0
	BEGIN
		SET @Success=0
		SET @Result = 0
		SET @Message = 'Unable to complete the action'
	END
ELSE BEGIN
	BEGIN
		SET @Success=1
		SET @Result = 1
		SET @Message = 'Action completed sucsessfully'
	END
END

SELECT @Result AS [Result], @Message AS [Message], @Success AS [Success]


Search Employee

In [ ]:
select * from MI_Employer_Employee where FullName like '%hubna%' and Deleted=0
select * from MI_Employer_Employee where Number like '612001'

Check Attendance data by from client

In [ ]:
select count(*) from MI_ONTIME_Attendance where (InTime !='1901-01-01' OR OutTime!='1901-01-01' ) 
and EmployeeCode in (
select distinct EmployeeCode from MI_Employer_Employee where Level2Code=1 AND DELETED=0
and textcardnumber in (
SELECT distinct TextCardNumber FROM MI_ONTIME_Log where TextCardNumber  in (select distinct TextCardNumber from MI_Employer_Employee where Level2Code=1 AND DELETED=0) and date='2021-02-02'
union
SELECT distinct TextCardNumber FROM MI_ONTIME_BackLog where TextCardNumber  in (select distinct TextCardNumber from MI_Employer_Employee where Level2Code=1 AND DELETED=0) and date='2021-02-02'
) )
and date='2021-02-01'

**Excel Quries**

For sql formate

- \=",'"&C2&"'" 

For update leaeves by number

- \="update MI\_ONTIME\_LeaveAllocation set Allocated="&H1712&" where LeaveYear=2021 and Deleted=0 and LeaveTypeCode=1 and EmployeeCode=(select EmployeeCode from MI\_Employer\_Employee where number='"&C1712&"' and Deleted=0)"

insert leaves into LeaveAllocation and Leave Balances excel

In [ ]:
	insert into MI_ONTIME_LeaveAllocation  values  ( 1,(select employeecode from MI_Employer_Employee where TextCardNumber=''), 2021,al,0,0,1,1,getdate(),0,'1901-01-01',0,'1901-01-01')
="insert into Leavebalances values (1,2021,0,getdate(),"&K29&",0,(select EmployeeCode from MI_Employer_Employee where Number='"&B29&"' and deleted=0))"

Leaves update from excel

In [ ]:
DECLARE @emp_id int  
DECLARE @annual decimal(18,2)  

DECLARE emp_cursor CURSOR FOR     
select  MI_Employer_Employee.EmployeeCode, annual from Sheet1 
inner join MI_Employer_Employee on MI_Employer_Employee.Number = Sheet1.id and Deleted=0 
where Annual!=0 and id is not null    
  
OPEN emp_cursor    
  
FETCH NEXT FROM emp_cursor INTO @emp_id ,@annual  
  
WHILE @@FETCH_STATUS = 0    
BEGIN    
 if not exists (select * from mi_ontime_leaveallocation where EmployeeCode=@emp_id and LeaveTypeCode=1 and LeaveYear=2021) 
	and @annual>0 and @emp_id >0
 begin 
	insert into MI_ONTIME_LeaveAllocation values (1,@emp_id,2021,@annual,0,0,1,1,getdate(),1,'1901-01-01',1,'1901-01-01') 
	end
      
    FETCH NEXT FROM emp_cursor     
INTO @emp_id ,@annual
   
END     
CLOSE emp_cursor;    
DEALLOCATE emp_cursor;
--44593
select count(*) from MI_ONTIME_LeaveAllocation
select distinct id from Sheet1 
where Annual!=0 and id is not null
delete from MI_ONTIME_LeaveAllocation where LeaveAllocationCode>55707 --order by CreatedDate desc
select top 1000 * from MI_ONTIME_LeaveAllocation where LeaveAllocationCode>55707
select getdate()

insert leaves by query using number and allocated amount

In [ ]:
Declare @emp_code int=0
set @emp_code =(select EmployeeCode,* from MI_Employer_Employee where Number='631403' and deleted=0) 

if not exists (select * from mi_ontime_leaveallocation where EmployeeCode=@emp_code and LeaveTypeCode=1 and LeaveYear=2021) 
begin 
insert into MI_ONTIME_LeaveAllocation values (1,@emp_code,2021,28,0,0,1,1,getdate(),1,'1901-01-01',1,'1901-01-01')
end

Get Dublicates

In [ ]:
select FullName,Number, TextCardNumber, NicNumber, EpfNo,
MI_EPRF_OrgLevel2.Name level2,
MI_EPRF_OrgLevel3.Name level3,
MI_EPRF_OrgLevel4.Name level4,
MI_EPRF_OrgLevel5.Name level5,
MI_EPRF_OrgLevel6.Name level6,
MI_EPRF_OrgLevel7.Name level7,
MI_EPRF_OrgLevel8.Name level8
from MI_Employer_Employee
left join MI_EPRF_OrgLevel2 on MI_EPRF_OrgLevel2.Level2Code = MI_Employer_Employee.Level2Code
left join MI_EPRF_OrgLevel3 on MI_EPRF_OrgLevel3.Level3Code = MI_Employer_Employee.Level3Code
left join MI_EPRF_OrgLevel4 on MI_EPRF_OrgLevel4.Level4Code = MI_Employer_Employee.Level4Code
left join MI_EPRF_OrgLevel5 on MI_EPRF_OrgLevel5.Level5Code = MI_Employer_Employee.Level5Code
left join MI_EPRF_OrgLevel6 on MI_EPRF_OrgLevel6.Level6Code = MI_Employer_Employee.Level6Code
left join MI_EPRF_OrgLevel7 on MI_EPRF_OrgLevel7.Level7Code = MI_Employer_Employee.Level7Code
left join MI_EPRF_OrgLevel8 on MI_EPRF_OrgLevel8.Level8Code = MI_Employer_Employee.Level8Code
where TextCardNumber !='' and  TextCardNumber in (select TextCardNumber from MI_Employer_Employee 
where Deleted=0 
group by TextCardNumber
having count(TextCardNumber)>1) 
order by TextCardNumber

Leaves work LTM

In [ ]:
insert into MI_ONTIME_LeaveAllocation 
SELECT 1,EmployeeCode, 2021,Allocated,0,0,1,0,getdate(),0,'1901-01-01',0,'1901-01-01' FROM Leavebalances  WHERE Deleted=0 --AND EmployeeCode 
--NOT IN (
--select distinct employeecode  from	MI_Employer_Employee  where EmploymentTypecode=1 and deleted=0 and datediff(MONTH, DateOfAppointment,getdate()) >=12
--and cast(getdate() as date)>=concat(SELECT * FROM LeavesAllocatedHistory WHERE DELETED=0
insert LeavesAllocatedHistory year(getdate()),'-',concat(REPLICATE('0', 2-LEN(month(Dateofappointment))),month(Dateofappointment)),'-',concat(REPLICATE('0', 2-LEN(day(dateofappointment))),day(dateofappointment))))  
update LeavesAllocatedHistory set Deleted=1, UpdatedDate=getdate() where Deleted=0 and EmployeeCode in 
(select EmployeeCode  from	MI_Employer_Employee  where EmploymentTypecode=1 and deleted=0 and datediff(MONTH, DateOfAppointment,getdate()) >=12
and concat(year(getdate()),'-',concat(REPLICATE('0', 2-LEN(month(Dateofappointment))),month(Dateofappointment)),'-',concat(REPLICATE('0', 2-LEN(day(dateofappointment))),day(dateofappointment))) <= '2021-03-04'
and  EmployeeCode not in (select EmployeeCode from MI_ONTIME_LeaveAllocation where LeaveTypeCode=1 and Deleted=0) and
EmployeeCode in (select EmployeeCode from LeavesAllocatedHistory where  deleted=0 ))
select * from Leavebalances where EmployeeCode=2188
select * from Leavebalances where EmployeeCode=1901

UPDATE LeavesAllocatedHistory SET Deleted=1, UpdatedDate=GETDATE() WHERE Deleted=0  
update MI_ONTIME_LeaveAllocation set Deleted=0 where DeletedDate > '2021-02-12 13:43:07.630' and Deleted=1 order by DeletedDate 
UPDATE MI_ONTIME_LeaveAllocation set Deleted=1, DeletedDate=GETDATE() where LeaveTypeCode=1 and Deleted=0
SELECT * FROM MI_ONTIME_LeaveAllocation where LeaveTypeCode=1 and Deleted=0
select * from  Leavebalances where deleted=0 AND EmployeeCode=

select COUNT(*) from MI_ONTIME_LeaveAllocation where deleted=0 AND LeaveTypeCode=3

select COUNT(*) from LeavesAllocatedHistory where deleted=0

select * from mi_ontime_leaveallocation where employeecode=2495 AND Deleted=0
Select Allocated from dbo.MI_ONTIME_LeaveAllocation where EmployeeCode = 2495 and LeaveYear = 2021 and LeaveTypeCode = 1 AND Deleted=0

select * from MI_ONTIME_LeaveAllocation where EmployeeCode in (
select employeecode  from	MI_Employer_Employee  where DateOfAppointment!='1901-01-01 00:00:00.000' AND datediff(MONTH, DateOfAppointment,getdate()) >=12
and YEAR(DateOfAppointment) <2020 AND MONTH(DateOfAppointment) < 2 AND MI_Employer_Employee.EmploymentTypeCode=1
) and Deleted=0 and LeaveTypeCode=1
select dateofappointment  from	MI_Employer_Employee WHERE (dateofappointment IS NULL OR dateofappointment <'2000-01-01')


select Allocated, Used,EmployeeCode from MI_ONTIME_LeaveAllocation where LeaveTypeCode=1 and Deleted=0 and 
EmployeeCode in (select EmployeeCode from Leavebalances where Deleted=0 and LeaveTypeCode=1 )

insert into MI_ONTIME_LeaveAllocation 
SELECT datediff(year,e.DateOfBirth,getdate()),e.DateOfAppointment,e.EmploymentTypeCode ,1,Leavebalances.EmployeeCode, 2021,Allocated,0,0,1,0,getdate(),0,'1901-01-01',0,'1901-01-01' FROM Leavebalances 
inner join MI_Employer_Employee e on e.EmployeeCode = Leavebalances.EmployeeCode
where Leavebalances.Deleted=0 and LeaveTypeCode=1 and Leavebalances.EmployeeCode not in (
select EmployeeCode from MI_ONTIME_LeaveAllocation where LeaveTypeCode=1 and Deleted=0)


insert into MI_ONTIME_LeaveAllocation 
SELECT 1,EmployeeCode, 2021,Allocated,0,0,1,0,getdate(),0,'1901-01-01',0,'1901-01-01' from Leavebalances where Deleted=0 and LeaveTypeCode=1 and EmployeeCode not in (
select EmployeeCode from MI_ONTIME_LeaveAllocation where LeaveTypeCode=1 and Deleted=0
)
select distinct employeecode from MI_ONTIME_LeaveAllocation where LeaveTypeCode=7 and UpdatedBy !=-1
update MI_ONTIME_LeaveAllocation set  Deleted=0, UpdatedDate=GETDATE() where LeaveTypeCode=7  and deleted=1 
select * from MI_ONTIME_LeaveType
select EmployeeCode, FULLNAME, EPFNO  from MI_Employer_Employee where EPFNO='' and Deleted=0
select EmployeeCode, FULLNAME from MI_Employer_Employee where Number='608932' and Deleted=0
select * from MI_ONTIME_LeaveAllocation where EmployeeCode=2029 and Deleted=0 and LeaveTypeCode=1
SELECT * FROM Leavebalances WHERE EmployeeCode=10927 AND Deleted=0 AND LeaveTypeCode=1


select * from mi_employer_employee where 

Man Power budget update

In [ ]:
select * from MI_OD_BudgetManPower
update MI_OD_BudgetManPower set Deleted=1, UpdatedBy=-1, UpdatedDate=getdate() where Deleted=0
DECLARE @emp_id		int
DECLARE @level1code int
DECLARE @level2code int
DECLARE @level3code int
DECLARE @level4code int
DECLARE @level5code int
DECLARE @level6code int
DECLARE @level7code int
DECLARE @level8code int
DECLARE @level9code int
DECLARE @level10code int
DECLARE @designationcode int
 

DECLARE emp_cursor CURSOR FOR
select EmployeeCode,level1code,Level2Code,Level3Code,level4code,Level5Code,Level6Code,Level7Code,Level8Code,Level9Code
,Level10Code,DesignationCode
from   MI_Employer_Employee where 	 Deleted=0
 
	 

OPEN emp_cursor

FETCH NEXT FROM emp_cursor INTO
 @emp_id		
,@level1code 
,@level2code 
,@level3code 
,@level4code 
,@level5code 
,@level6code 
,@level7code 
,@level8code 
,@level9code 
,@level10code
,@designationcode

WHILE @@FETCH_STATUS = 0
BEGIN
	if not exists (select *
		from MI_OD_BudgetManPower
		where
    level1code = @level1code 
and level2code = @level2code 
and level3code = @level3code 
and level4code = @level4code 
and level5code = @level5code 
and level6code = @level6code 
and level7code = @level7code 
and level8code = @level8code 
and level9code = @level9code 
and level10code = @level10code
and designationcode = @designationcode
and  Deleted=0

			)
		
	begin
		insert into MI_OD_BudgetManPower
		values ( @level1code 
,@level2code 
,@level3code 
,@level4code 
,@level5code 
,@level6code 
,@level7code 
,@level8code 
,@level9code 
,@level10code
,0
,@designationcode, 1,1,0, 0, 1, 1, getdate(), 0, '1901-01-01', 0, '1901-01-01' )
	end
	else
	begin
		update MI_OD_BudgetManPower
		set BudgetHeadCount = BudgetHeadCount+1--,RequestedBudget=RequestedBudget+1
		where 
		    level1code = @level1code 
and level2code = @level2code 
and level3code = @level3code 
and level4code = @level4code 
and level5code = @level5code 
and level6code = @level6code 
and level7code = @level7code 
and level8code = @level8code 
and level9code = @level9code 
and level10code = @level10code
and designationcode = @designationcode
and  Deleted=0
	end

FETCH NEXT FROM emp_cursor INTO
 @emp_id		
,@level1code 
,@level2code 
,@level3code 
,@level4code 
,@level5code 
,@level6code 
,@level7code 
,@level8code 
,@level9code 
,@level10code
,@designationcode

END
CLOSE emp_cursor;
DEALLOCATE emp_cursor;